# Ya ML CV HW 6

In [ ]:
# загрузили архив https://disk.yandex.ru/d/ECLHqVyX-F5hRA
!mkdir -p my_folder
!tar -xvzf best_pictures.tar.gz -C my_folder

In [ ]:
import pandas as pd
import numpy as np

# Считываем
filters_df = pd.read_csv('my_folder/algos.csv', header=None)

# Превращаем строки в 3×3 фильтры
filter1 = filters_df.iloc[0].values.reshape((3, 3))
filter2 = filters_df.iloc[1].values.reshape((3, 3))

print("Фильтр 1:\n", filter1)
print("Фильтр 2:\n", filter2)


Фильтр 1:
 [[-1.  -0.5  0. ]
 [-0.5  0.5  0.5]
 [ 0.   0.5  1. ]]
Фильтр 2:
 [[0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.0625]]


In [ ]:
# Третий фильтр, например, размытие (усреднение)
filter3 = np.array([[1,1,1],
                    [1,1,1],
                    [1,1,1]]) / 9


In [ ]:
from scipy.ndimage import convolve
from PIL import Image

# Загружаем изображение в ч/б
img = Image.open('my_folder/1.png').convert('L')
img_array = np.array(img)

# Применяем фильтры по очереди
out1 = convolve(img_array, filter1, mode='reflect')
out2 = convolve(out1, filter2, mode='reflect')
out3 = convolve(out2, filter3, mode='reflect')

# Сохраним результат чтобы посмотреть
Image.fromarray(np.clip(out3, 0, 255).astype(np.uint8)).save('output.png')


In [ ]:
# Читаем эталон из txt
ref = np.loadtxt('my_folder/1.txt')

# Сравниваем
difference = np.abs(ref - out3)
print("Средняя ошибка:", np.mean(difference))
print("Максимальная ошибка:", np.max(difference))


Средняя ошибка: 47.871323585510254
Максимальная ошибка: 342.44921875


In [ ]:
import itertools
import numpy as np
from scipy.ndimage import convolve
from PIL import Image
import pandas as pd

# 1. Загружаем фильтры из CSV
filters_df = pd.read_csv('my_folder/algos.csv', header=None)
filter1 = filters_df.iloc[0].values.reshape((3, 3))
filter2 = filters_df.iloc[1].values.reshape((3, 3))

# 2. Загружаем картинку и эталон
img = Image.open('my_folder/1.png').convert('L')
img_array = np.array(img)

ref = np.loadtxt('my_folder/1.txt')

# 3. Функция для теста комбинации фильтров
def test_filters(filters, order, img_array, ref):
    temp = img_array.copy()
    for idx in order:
        temp = convolve(temp, filters[idx], mode='reflect')
    difference = np.abs(ref - temp)
    mean_error = np.mean(difference)
    return mean_error, temp

# 4. Автоматический перебор
best_error = float('inf')
best_order = None
best_filter3 = None
best_output = None

# Генерируем разные варианты третьего фильтра
# (можно сделать более умную генерацию позже)
possible_values = [-1, 0, 1, 2]  # значения для ячеек фильтра
all_possible_filters3 = []

for vals in itertools.product(possible_values, repeat=9):
    f3 = np.array(vals).reshape((3, 3))
    all_possible_filters3.append(f3)

print(f"Всего вариантов третьего фильтра: {len(all_possible_filters3)}")

# Перебираем все варианты
for f3 in all_possible_filters3:
    filters = [filter1, filter2, f3]

    for order in itertools.permutations([0, 1, 2]):
        mean_error, temp = test_filters(filters, order, img_array, ref)

        if mean_error < best_error:
            best_error = mean_error
            best_order = order
            best_filter3 = f3.copy()
            best_output = temp.copy()

print("\n=== Результаты ===")
print("Лучшая ошибка:", best_error)
print("Лучший порядок фильтров:", best_order)
print("Лучший третий фильтр:\n", best_filter3)

# Сохраняем лучший результат
Image.fromarray(np.clip(best_output, 0, 255).astype(np.uint8)).save('best_output.png')


Всего вариантов третьего фильтра: 262144


KeyboardInterrupt: 

In [ ]:
# НЕЙРОНКОЙ ПОДБЕРЕМ ФИЛЬТР
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import numpy as np
import pandas as pd
import glob

# Загружаем фильтры из CSV
filters_df = pd.read_csv('my_folder/algos.csv', header=None)
filter1 = torch.tensor(filters_df.iloc[0].values.reshape((3, 3)), dtype=torch.float32)
filter2 = torch.tensor(filters_df.iloc[1].values.reshape((3, 3)), dtype=torch.float32)

# Фиксированные фильтры для свёртки
fixed_filters = [filter2, filter1]

# Собираем все файлы в папке
image_files = sorted(glob.glob('my_folder/*.png'))
txt_files = sorted(glob.glob('my_folder/*.txt'))

dataset = []
for img_path, txt_path in zip(image_files, txt_files):
    img = Image.open(img_path).convert('L')
    img_array = np.array(img, dtype=np.float32) #/ 255.0  # нормализация
    target_array = np.loadtxt(txt_path).astype(np.float32) #/ 255.0  # нормализация

    dataset.append((torch.tensor(img_array).unsqueeze(0), torch.tensor(target_array).unsqueeze(0)))  # (1, H, W)

class LearnableFilter(nn.Module):
    def __init__(self):
        super().__init__()
        # Инициализируем третий фильтр случайными числами
        self.learned_filter = nn.Parameter(torch.randn(1, 1, 3, 3))  # 1 входной канал, 1 выходной канал, 3x3

    def forward(self, x):
        # Применяем фиксированные фильтры (ручная свёртка)
        for filt in fixed_filters[1:]:
            filt = filt.unsqueeze(0).unsqueeze(0)  # (out_channels, in_channels, H, W)
            x = nn.functional.conv2d(x, filt, padding=1)

        # Применяем обучаемый фильтр
        x = nn.functional.conv2d(x, self.learned_filter, padding=1)

        # Применяем фиксированные фильтры (ручная свёртка)
        for filt in fixed_filters[:1]:
            filt = filt.unsqueeze(0).unsqueeze(0)  # (out_channels, in_channels, H, W)
            x = nn.functional.conv2d(x, filt, padding=1)

        return x

model = LearnableFilter()
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

n_epochs = 100

for epoch in range(n_epochs):
    total_loss = 0.0
    for img_tensor, target_tensor in dataset:
        img_tensor = img_tensor.unsqueeze(0)  # добавляем batch размерность
        target_tensor = target_tensor.unsqueeze(0)

        output = model(img_tensor)
        loss = loss_fn(output, target_tensor)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if (epoch+1) % 1 == 0 or epoch == 0:
        print(f"Эпоха {epoch+1}, Loss: {total_loss/len(dataset)}")

print("\nОбучение завершено!")
print("Обученный третий фильтр:")
print(model.learned_filter.detach().squeeze().numpy())

Эпоха 1, Loss: 1254.842674058914
Эпоха 2, Loss: 952.0612042121887
Эпоха 3, Loss: 936.7474290885925
Эпоха 4, Loss: 930.0377954788208
Эпоха 5, Loss: 927.0875525684356
Эпоха 6, Loss: 925.5423328304291
Эпоха 7, Loss: 924.5120725803375
Эпоха 8, Loss: 923.6857079887391
Эпоха 9, Loss: 922.9555663642883
Эпоха 10, Loss: 922.2834357376098
Эпоха 11, Loss: 921.6548242835999
Эпоха 12, Loss: 921.0633903121948
Эпоха 13, Loss: 920.5056728458404
Эпоха 14, Loss: 919.9793096599578
Эпоха 15, Loss: 919.4823542613983
Эпоха 16, Loss: 919.0131056175231
Эпоха 17, Loss: 918.5700126514434
Эпоха 18, Loss: 918.1515630645752
Эпоха 19, Loss: 917.7564108867646
Эпоха 20, Loss: 917.3832849788665
Эпоха 21, Loss: 917.030924150467
Эпоха 22, Loss: 916.6981822738647
Эпоха 23, Loss: 916.3839755363464
Эпоха 24, Loss: 916.0872711715698
Эпоха 25, Loss: 915.8071054039001
Эпоха 26, Loss: 915.5425574855805
Эпоха 27, Loss: 915.292736995697
Эпоха 28, Loss: 915.0568533267975
Эпоха 29, Loss: 914.8341176395417
Эпоха 30, Loss: 914.62380

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
import glob

def evaluate_model(model, image_folder='my_folder'):
    image_files = sorted(glob.glob(f'{image_folder}/*.png'))
    txt_files = sorted(glob.glob(f'{image_folder}/*.txt'))

    mse_losses = []

    for img_path, txt_path in zip(image_files, txt_files):
        # Загрузка изображения (без нормализации!)
        img = Image.open(img_path).convert('L')
        img_tensor = torch.tensor(np.array(img, dtype=np.float32)).unsqueeze(0).unsqueeze(0)  # (B, C, H, W)

        # Загрузка целевого выхода
        target_array = np.loadtxt(txt_path).astype(np.float32)
        target_tensor = torch.tensor(target_array).unsqueeze(0).unsqueeze(0)  # (B, C, H, W)

        # Прогон через модель
        output = model(img_tensor)

        # Подсчёт MSE для этой пары
        mse = F.mse_loss(output, target_tensor).item()
        mse_losses.append(mse)

    # Среднее значение MSE
    avg_mse = np.mean(mse_losses)
    print(f"Среднее MSE на всех данных: {avg_mse:.6f}")
    return avg_mse

evaluate_model(model)

Среднее MSE на всех данных: 0.000000


np.float64(6.252211647329864e-10)

In [ ]:
# Готовим всё для отправки

# 1. Фиксированные фильтры
filter1 = filters_df.iloc[0].values.reshape((3, 3))
filter2 = filters_df.iloc[1].values.reshape((3, 3))

# 2. Обученный фильтр
learned_filter = model.learned_filter.detach().squeeze().numpy()

# 3. Собираем всё в одну кучу
all_filters = [filter1, filter2, learned_filter]

# 4. Готовим строки для отправки
rows = []
for f in all_filters:
    row = ','.join([f"{x:.4f}" for x in f.flatten()])
    rows.append(row)

# 5. Объединяем
submission = '\n'.join(rows)

print(submission)

-1.0000,-0.5000,0.0000,-0.5000,0.5000,0.5000,0.0000,0.5000,1.0000
0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625
0.1250,0.2500,0.1250,0.2500,0.5000,0.2500,0.1250,0.2500,0.1250


1 2 3
Loss: 889.9

2 1 3
Loss: 43.8

3 1 2
Loss: 515.0

3 2 1
Loss: 4.5137745437906496e-07

1 3 2
Loss: 1.417360968758441e-09

2 3 1
Loss: 913.5866336116791